# LANE DRIVING IN TOWN 01
María Rodríguez Palomo

In [1]:
import glob
import os
import sys
import carla
import argparse
import random
import time
import math
import numpy as np
import cv2

#### WORLD CREATION

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(2000)
client.load_world("Town01")
world = client.get_world()
map = world.get_map()

#### VIEW CHANGE

In [3]:
start_point=carla.Transform(carla.Location(x=2.059280, y=134.749863, z=2.300000), carla.Rotation(pitch=0.000000, yaw=269.999756, roll=0.000000))
spectator = world.get_spectator()
#TOWN01
spectator_pos = carla.Transform(start_point.location + carla.Location(x=0,y=10,z=4),carla.Rotation(yaw=start_point.rotation.yaw))
spectator.set_transform(spectator_pos)

#### VEHICLE CREATION

In [4]:
blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('vehicle.tesla.model3'))

if bp.has_attribute('color'):
    color = bp.get_attribute('color').recommended_values[2]
    bp.set_attribute('color', '255,0,255')

vehicle = world.spawn_actor(bp, start_point)

#### CONSTANT VELOCITY

In [5]:
def maintain_speed(s,preferred_speed,speed_threshold):
    if s>=preferred_speed:
        return 0
    elif s < preferred_speed - speed_threshold:
        return 0.8
    else:
        return 0.4

#### ANGLE

In [6]:
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

def get_angle(car,wp):
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))
    return degrees

In [7]:
def steer(predicted_angle,MAX_STEER_DEGREES):
    if predicted_angle<-180:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 180:
        predicted_angle = predicted_angle-360
    steer_input = predicted_angle
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    steer_input = steer_input/75
    return steer_input

#### DEVELOPER VIEW

In [8]:
def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

In [9]:
CAMERA_POS_Z = 3
CAMERA_POS_X = -5
camera_bp=world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x','640')
camera_bp.set_attribute('image_size_y','360')
camera_init_trans= carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
camera=world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
image_w= camera_bp.get_attribute('image_size_x').as_int()
image_h= camera_bp.get_attribute('image_size_y').as_int()
camera_data = {'image':np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

#### DRIVING

In [10]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
quit=False
i=0 #cada 50 veces que entre al bucle se cambia el clima
preferred_speed = 30
speed_threshold = 2
MAX_STEER_DEGREES = 50

while True:
    #CAMARA 
    world.tick()
    if cv2.waitKey(1)==ord('q'): #si pulsamos la tecla q la pestaña desaparece
        quit=True
        break
    image = camera_data['image']
        
    loc=vehicle.get_location()
    current_w = map.get_waypoint(loc)
    waypoint_separation = 5
    next_w0 = list(current_w.next(waypoint_separation))[0]
    next_w = map.get_waypoint(next_w0.transform.location,project_to_road=True, lane_type=(carla.LaneType.Driving))

    velocity = vehicle.get_velocity()
    speed = round(3.6 *math.sqrt(velocity.x**2+ velocity.y**2+velocity.z**2),0)
    image=cv2.putText(image,'Speed: '+str(int(speed))+' km/h', (30,50),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1,cv2.LINE_AA)
    estimated_throttle = maintain_speed(speed,preferred_speed,speed_threshold)
    predicted_angle = get_angle(vehicle,next_w)
    steer_input=steer(predicted_angle,MAX_STEER_DEGREES)
    if steer_input > 0.035:
        estimated_throttle=0
    vehicle.apply_control(carla.VehicleControl(throttle= estimated_throttle, steer=steer_input))
    
    cv2.imshow('RGB Camera', image)
cv2.destroyAllWindows()
camera.stop()

#### DESTROY

In [11]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()